## What is an event?
Event is a teamfight, that is when players start fighting.

Each row will be an event.

Each game will consist of several events.

## What is y for an event?
Dependent variable will be a gold difference between teams

## What is a variable of interest?
Being toxic. I will have a dummy for each player being toxic. The variable of interest may be:
* team 1 toxic and/or team 2 toxic.
* sum of toxics in a team (0 to 5)

## What are X for an event?
Those include gold, last hits (# of kills of non-player characters (NPC)), xp (experience that you get from kills), player rating (based on history of games). 

Note that player rating is game specific, that is same for all events in the game  

In [1]:
import pandas as pd
import numpy as np

In [2]:
# teamfights data for each player 
tf_player = pd.read_csv('../../Data/Original/teamfights_players.csv')

# teamfights timings
tf = pd.read_csv('../../Data/Original/teamfights.csv')

# player info for each minute
player_time = pd.read_csv('../../Data/Original/player_time.csv')

# chat data
chat = pd.read_csv('../../Data/Original/chat.csv')

# player ratings
player_ratings = pd.read_csv('../../Data/Original/player_ratings.csv')

# player info for the match
player = pd.read_csv('../../Data/Original/players.csv')

In [26]:
tf.head()

,match_id,start,end,last_death,deaths
0,0,220,252,237,3
1,0,429,475,460,3
2,0,900,936,921,3
3,0,1284,1328,1313,3
4,0,1614,1666,1651,5


Start time of each teamfight will identify the events. So, for match 0, event 0 is from time 0 to 220, event 1 is from time 220 to 429, etc.

In [4]:
# number of games under analysis
n_observations = 100 

### Add a start column to tf_player dataframe

In [18]:
# create empty 
tf_player["start"] = ""

# event_times for games
event_times = tf.start

In [21]:
def n_times(v,n):
    for i in range(n):
        yield v

event_times = [i for j in event_times for i in n_times(j,10)]

In [24]:
tf_player["start"] = np.array(event_times)

In [30]:
tf_player['radiant'] = np.where(tf_player['player_slot']<= 5, 1, 0)

### Get sum of gold by each team

In [34]:
x = tf_player.groupby(['match_id', 'start', 'radiant']).sum()['gold_delta']

In [56]:
radiant_gold = x.loc[:,:,1]
dire_gold = x.loc[:,:,0]

In [59]:
tf['radiant_gold'] = np.array(radiant_gold)
tf['dire_gold'] = np.array(dire_gold)

In [61]:
tf['delta_gold'] = tf['radiant_gold'] - tf['dire_gold']

In [64]:
tf.head()

,match_id,start,end,last_death,deaths,radiant_gold,dire_gold,delta_gold
0,0,220,252,237,3,632,790,-158
1,0,429,475,460,3,1929,345,1584
2,0,900,936,921,3,3145,-506,3651
3,0,1284,1328,1313,3,1872,-112,1984
4,0,1614,1666,1651,5,1768,2164,-396


### Add player in-game characteristics to the tf

In [174]:
# round start time by minute
tf['times'] = (tf.start//60)*60
tf['times']= np.where(tf['times']< 0, 0, tf['times'])

In [182]:
tf2 = pd.merge(tf, player_time,  how='left', left_on=['match_id','times'], right_on = ['match_id','times'])

In [185]:
tf2.head()

,match_id,start,end,last_death,deaths,radiant_gold,dire_gold,delta_gold,times,gold_t_0,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,220,252,237,3,632,790,-158,180,683,...,842,958,16,1135,300,0,210,1290,8,527
1,0,429,475,460,3,1929,345,1584,420,1257,...,2033,2652,47,3111,700,0,241,3033,28,1329
2,0,900,936,921,3,3145,-506,3651,900,3324,...,4472,6534,114,7788,1767,6,1486,6470,60,3822
3,0,1284,1328,1313,3,1872,-112,1984,1260,5019,...,5583,8777,147,9713,2606,8,1929,9637,87,6372
4,0,1614,1666,1651,5,1768,2164,-396,1560,6394,...,6892,11091,171,12333,3678,15,3392,12017,108,8150


### Add chat to tf2

In [219]:
# change slot accordingly
chat.loc[chat['slot'] ==5 , 'slot'] = 128
chat.loc[chat['slot'] ==6 , 'slot'] = 129
chat.loc[chat['slot'] ==7 , 'slot'] = 130
chat.loc[chat['slot'] ==8 , 'slot'] = 131
chat.loc[chat['slot'] ==9 , 'slot'] = 132

In [254]:
for i in range(50000):
    start = tf2[tf2.match_id == i]['start'].to_list()
    for k in range(len(start)):
        chat.loc[(chat['match_id']==i)& (chat['time']<start[len(start)-k-1]), 'start'] = start[len(start)-k-1]

KeyboardInterrupt: 

### Only first 20,000 games

In [264]:
chat = chat[chat.match_id<20000]

In [266]:
chat = chat.dropna()

In [270]:
chat = chat.groupby(['match_id', 'start', 'slot'])['key'].apply(' '.join).reset_index()

In [280]:
chat.head()

,match_id,start,slot,key
0,0,220.0,1,space created hah
1,0,220.0,129,force it ez 500
2,0,1284.0,4,mvp ulti
3,0,1614.0,0,fuck my ass
4,0,1614.0,1,hah


### Predict toxicity

In [282]:
import pickle

In [283]:
# load model
model = pickle.load(open('../toxic_classifier/log_model.pkl', 'rb'))

In [284]:
import os
import re
import nltk
import string
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, LancasterStemmer 

from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC

import warnings
warnings.filterwarnings("ignore")

In [285]:
# load the stopword list provided by the NLTK library
stop_words = stopwords.words('english')

In [292]:
def preprocessing_text(text):
    text = re.sub(r'[^A-Za-z0-9 ]+', ' ', text) #remove all non‐alphanumeric characters except white space
    text = re.sub("(.)\\1{2,}", "\\1", text) #replace 3+ consecutive letters to 1 (loooovvvve -> love)
    text = re.sub(r'([a-z0-9+._-]+@[a-z0-9+._-]+\.[a-z0-9+_-]+)',"", text) # remove emails
    text = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , text) # remove urls
    words = word_tokenize(text.lower())
    tokens = [word for word in words if word not in stop_words]
    tokens = [token for token in tokens if not any(c.isdigit() for c in token)] #remove everything containing digits
    lemmatizer = WordNetLemmatizer()
    tokens_lematized = [lemmatizer.lemmatize(word) for word in tokens]
    preprocessed_text = ' '.join(tokens_lematized)
    return preprocessed_text

In [294]:
with open("../toxic_classifier/comment_data.pkl", "rb") as fp:   # Unpickling
    df = pickle.load(fp)

In [295]:
vectorizer = TfidfVectorizer() 
tfidf = vectorizer.fit_transform(df.comment_text)

In [296]:
def get_pred(s):
    x = []
    x.append(s)
    x[0] = preprocessing_text(x[0])
    vec = vectorizer.transform(x)
    label = model.predict(vec)[0]
    return label

In [299]:
chat['toxic'] = chat['key'].apply(lambda x: get_pred(x))

In [300]:
chat

,match_id,start,slot,key,toxic
0,0,220.0,1,space created hah,0
1,0,220.0,129,force it ez 500,0
2,0,1284.0,4,mvp ulti,0
3,0,1614.0,0,fuck my ass,1
4,0,1614.0,1,hah,0
...,...,...,...,...,...
177255,19998,2358.0,132,ой чет сьебало У меня а это все норм Парни эбм...,0
177256,19998,2427.0,0,nice linkens idiot,1
177257,19998,2427.0,3,LOL EMBER lose again,0
177258,19998,2427.0,132,чет вам похуй_)) потому что эмбер ебоиит),0


In [298]:
chat

,match_id,start,slot,key
0,0,220.0,1,space created hah
1,0,220.0,129,force it ez 500
2,0,1284.0,4,mvp ulti
3,0,1614.0,0,fuck my ass
4,0,1614.0,1,hah
...,...,...,...,...
177255,19998,2358.0,132,ой чет сьебало У меня а это все норм Парни эбм...
177256,19998,2427.0,0,nice linkens idiot
177257,19998,2427.0,3,LOL EMBER lose again
177258,19998,2427.0,132,чет вам похуй_)) потому что эмбер ебоиит)


In [279]:
chat.pivot(columns='slot', values=['key'])

ValueError: Index contains duplicate entries, cannot reshape

In [273]:
chat

,match_id,start,slot,key
0,0,220.0,1,space created hah
1,0,220.0,129,force it ez 500
2,0,1284.0,4,mvp ulti
3,0,1614.0,0,fuck my ass
4,0,1614.0,1,hah
...,...,...,...,...
177255,19998,2358.0,132,ой чет сьебало У меня а это все норм Парни эбм...
177256,19998,2427.0,0,nice linkens idiot
177257,19998,2427.0,3,LOL EMBER lose again
177258,19998,2427.0,132,чет вам похуй_)) потому что эмбер ебоиит)


In [268]:
chat[chat.match_id==19999]

,match_id,key,slot,time,unit,start
580423,19999,????,128,203,"↖(●,,,●)↗",361.0


In [263]:
chat.to_csv('../../Data/Modified/chat_modified_v1.csv')

In [241]:
chat.loc[(chat['match_id']==0)& (chat['time']<start[len(start)-0-1]), 'start'] = start[len(start)-0-1]

In [258]:
tf2[tf2.match_id == 1000]

,match_id,start,end,last_death,deaths,radiant_gold,dire_gold,delta_gold,times,gold_t_0,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
10666,1000,630,668,653,3,989,-277,1266,600,2359,...,1750,2170,20,2321,2881,33,2974,1389,5,1740
10667,1000,668,724,709,4,1023,1864,-841,660,3085,...,1882,2311,21,2392,3029,34,3065,1529,6,1831
10668,1000,941,999,984,6,1011,306,705,900,4508,...,4295,3021,25,3147,4630,43,4460,2577,14,3275
10669,1000,1029,1080,1065,5,1504,527,977,1020,5212,...,4863,3467,27,3237,5151,43,5059,2937,14,3324
10670,1000,1283,1319,1304,3,144,1438,-1294,1260,7230,...,5572,4706,47,5089,6115,45,6307,3841,23,4674
10671,1000,1570,1603,1588,3,705,758,-53,1560,8924,...,6670,6503,63,7240,8402,54,8577,5320,26,5754
10672,1000,1696,1748,1733,5,1620,2066,-446,1680,9808,...,7343,7407,68,7891,8806,55,9153,6413,27,7681
10673,1000,1842,1902,1887,8,2553,-281,2834,1800,10487,...,8571,9019,69,9454,9627,56,9840,6914,27,8012
10674,1000,2054,2108,2093,8,5438,-522,5960,2040,14142,...,9229,10233,80,11550,10527,59,10403,7694,36,8741
10675,1000,2174,2217,2202,3,106,3493,-3387,2160,15082,...,14064,11205,95,12623,11494,65,11120,7894,36,9489


In [262]:
chat[chat.match_id==20000]

,match_id,key,slot,time,unit,start
580425,20000,LOL THIS INVOKER,2,1429,R-Dog,1519.0
580426,20000,LOL,2,1433,R-Dog,1519.0
580427,20000,SO BAD,2,1435,R-Dog,1519.0
580428,20000,AHAHAHUAHAH,2,1436,R-Dog,1519.0
580429,20000,tjis pudge,3,1670,Elanu,1742.0
580430,20000,he thinks is our fault cuz he can t hook,3,1681,Elanu,1742.0
580431,20000,pff dazzle flaming me all game,131,1696,Hajsine_!,1742.0
580432,20000,:D,131,1697,Hajsine_!,1742.0
580433,20000,gg,2,1779,R-Dog,1898.0
580434,20000,report pudge,1,1785,RZA,1898.0


In [228]:
start = tf2[tf2.match_id == 0]['start']

In [229]:
start[0]

220

In [232]:
chat

,match_id,key,slot,time,unit,start
0,0,force it,129,-8,6k Slayer,220.0
1,0,space created,1,5,Monkey,220.0
2,0,hah,1,6,Monkey,220.0
3,0,ez 500,129,9,6k Slayer,220.0
4,0,mvp ulti,4,934,Kira,NaN
...,...,...,...,...,...,...
1439483,49999,ez,131,2796,DamN.Pussy_Jr,NaN
1439484,49999,ya ya so ezx,1,2796,StopDancing,NaN
1439485,49999,ez,131,2797,DamN.Pussy_Jr,NaN
1439486,49999,hahaha,1,2798,StopDancing,NaN


In [231]:
chat.loc[(chat['match_id']==0)& (chat['time']<start[len(start)-0]), 'start'] = start[len(start)-0]

In [248]:
start

12     259
13     327
14     452
15     772
16     867
17     924
18    1203
19    1372
20    1431
21    1688
22    2049
23    2148
24    2318
25    2472
26    2563
Name: start, dtype: int64

In [247]:
start[0]

KeyError: 0

In [237]:
start[len(start)-1]

2351

In [239]:
for k in range(len(start)):
    print(start[len(start)-1-k])

2351
2248
2101
1863
1818
1734
1672
1614
1284
900
429
220


In [221]:
chat

,match_id,key,slot,time,unit,First Season
0,0,force it,129,-8,6k Slayer,129.0
1,0,space created,1,5,Monkey,NaN
2,0,hah,1,6,Monkey,NaN
3,0,ez 500,129,9,6k Slayer,129.0
4,0,mvp ulti,4,934,Kira,NaN
...,...,...,...,...,...,...
1439483,49999,ez,131,2796,DamN.Pussy_Jr,131.0
1439484,49999,ya ya so ezx,1,2796,StopDancing,NaN
1439485,49999,ez,131,2797,DamN.Pussy_Jr,131.0
1439486,49999,hahaha,1,2798,StopDancing,NaN


In [225]:
for i in range()

,match_id,key,slot,time,unit
0,0,force it,129,-8,6k Slayer
1,0,space created,1,5,Monkey
2,0,hah,1,6,Monkey
3,0,ez 500,129,9,6k Slayer
4,0,mvp ulti,4,934,Kira
...,...,...,...,...,...
1439483,49999,ez,131,2796,DamN.Pussy_Jr
1439484,49999,ya ya so ezx,1,2796,StopDancing
1439485,49999,ez,131,2797,DamN.Pussy_Jr
1439486,49999,hahaha,1,2798,StopDancing


In [213]:
chat[chat.match_id == 0]

,match_id,key,slot,time,unit
0,0,force it,6,-8,6k Slayer
1,0,space created,1,5,Monkey
2,0,hah,1,6,Monkey
3,0,ez 500,6,9,6k Slayer
4,0,mvp ulti,4,934,Kira
5,0,bye,6,1486,6k Slayer
6,0,hah,1,1488,Monkey
7,0,fate,6,1496,6k Slayer
8,0,is cruel,6,1502,6k Slayer
9,0,fuck my ass,0,1524,Double T


In [214]:
chat.slot.unique()

array([6, 1, 4, 0, 7, 9, 2, 3, 5, 8], dtype=int64)

In [207]:
chat[chat['slot']==-9]

,match_id,key,slot,time,unit
652287,22654,ggwp,-9,-450,Dana


In [209]:
chat.at[652287,'slot']=3

In [208]:
chat[chat.match_id == 22654]

,match_id,key,slot,time,unit
652287,22654,ggwp,-9,-450,Dana
652288,22654,wow,7,-58,(>PRETTY GAMER <)
652289,22654,wait,0,-42,'' Anonymous is Legend '' ?
652290,22654,:D,0,-42,'' Anonymous is Legend '' ?
652291,22654,w8 watch me?,8,-42,ComebackisreaL :DD
...,...,...,...,...,...
652474,22654,askjda',3,3136,Dana
652475,22654,yeah comback isreal,4,3139,Noah
652476,22654,iyak sa gem,0,3140,'' Anonymous is Legend '' ?
652477,22654,wahaha,0,3141,'' Anonymous is Legend '' ?


In [181]:
player_time

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,60,409,0,63,142,1,186,168,0,...,62,345,6,351,100,0,77,613,1,125
2,0,120,546,0,283,622,4,645,330,0,...,543,684,12,805,200,0,210,815,5,323
3,0,180,683,1,314,927,9,1202,430,0,...,842,958,16,1135,300,0,210,1290,8,527
4,0,240,956,1,485,1264,11,1583,530,0,...,1048,1500,26,1842,400,0,210,1431,9,589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209773,49999,2640,19098,215,20444,18261,129,27832,18988,89,...,24025,15992,91,16998,34374,296,32701,23114,290,22774
2209774,49999,2700,19460,227,20659,18478,134,28012,19108,90,...,24025,16092,91,16998,34474,296,32701,23214,290,22774
2209775,49999,2760,19948,237,20889,18889,148,28437,19208,90,...,25430,16518,91,17509,35434,305,32701,23602,290,24179
2209776,49999,2820,19997,237,20889,18938,148,28437,19257,90,...,28237,17246,92,18815,36272,306,32701,24181,294,24772


In [177]:
for i in columns_to_create:
    tf[i] = ""
    for k in range(len(tf)):
        tf.loc[k,i] = player_time.loc[player_time['match_id'] == tf.loc[k].match_id, ('times','lh_t_1')].loc[player_time['times'] == tf.loc[k].times, 'lh_t_1'].iloc[0]


KeyboardInterrupt: 

In [167]:
tf.loc[k].times

-60

In [166]:
tf.loc[k].match_id

23

In [161]:
tf.loc[0,i] = 2

In [ ]:
tf.loc[tf['A'] > 2, 'B'] = new_val

In [152]:
i

'gold_t_0'

In [171]:
tf[tf.match_id == 23]

,match_id,start,end,last_death,deaths,radiant_gold,dire_gold,delta_gold,times,gold_t_0
250,23,-10,34,19,5,864,1012,-148,-60,
251,23,175,216,201,3,309,924,-615,120,
252,23,397,434,419,3,-161,1245,-1406,360,
253,23,502,548,533,3,1276,1448,-172,480,
254,23,579,626,611,4,362,1403,-1041,540,
255,23,841,893,878,5,-325,2338,-2663,840,
256,23,1000,1052,1037,4,1567,637,930,960,
257,23,1291,1337,1322,5,1790,1079,711,1260,
258,23,2279,2332,2317,5,3903,-2017,5920,2220,
259,23,2349,2382,2367,3,7328,527,6801,2340,


In [173]:
tf.times.unique()

array([  180,   420,   900,  1260,  1560,  1620,  1680,  1800,  1860,
        2100,  2220,  2340,   240,   300,   720,   840,  1200,  1320,
        1380,  2040,  2280,  2460,  2520,   660,   780,   960,  1440,
        1920,  2400,  2580,  2640,   600,  1020,  1140,  1980,  2760,
        3000,  1500,  1080,    60,   540,  1740,  2700,  2880,  2940,
        3120,  3240,   480,  2160,  2820,  3180,  3300,  3480,  3540,
        3660,  3900,  3060,   -60,   120,   360,  3360,  3420,  4020,
        4200,  4260,  3720,  3780,  4140,  3600,  3960,  4320,  5160,
        5040,  5100,     0,  3840,  4440,  4080,  4380,  4500,  4740,
        4620,  4860,  4980,  5340,  4800,  5640,  5760,  5880,  4680,
        4920,  4560,  8640, 10080, 10260, 10500, 11040, 11460, 11700,
       11940, 12540, 12960, 13140, 13920, 14100, 14160, 14340, 14580,
       14940, 15060, 15300, 15360, 15420, 15540, 15900,  5280,  5400,
        5460,  5220,  5580,  6300,  6360,  6420,  6060,  6480,  6540,
        5520], dtype

In [172]:
player_time.times.unique()

array([    0,    60,   120,   180,   240,   300,   360,   420,   480,
         540,   600,   660,   720,   780,   840,   900,   960,  1020,
        1080,  1140,  1200,  1260,  1320,  1380,  1440,  1500,  1560,
        1620,  1680,  1740,  1800,  1860,  1920,  1980,  2040,  2100,
        2160,  2220,  2280,  2340,  2400,  2460,  2520,  2580,  2640,
        2700,  2760,  2820,  2880,  2940,  3000,  3060,  3120,  3180,
        3240,  3300,  3360,  3420,  3480,  3540,  3600,  3660,  3720,
        3780,  3840,  3900,  3960,  4020,  4080,  4140,  4200,  4260,
        4320,  4380,  4440,  4500,  4560,  4620,  4680,  4740,  4800,
        4860,  4920,  4980,  5040,  5100,  5160,  5220,  5280,  5340,
        5400,  5460,  5520,  5580,  5640,  5700,  5760,  5820,  5880,
        5940,  6000,  6060,  6120,  6180,  6240,  6300,  6360,  6420,
        6480,  6540,  6600,  6660,  6720,  6780,  6840,  6900,  6960,
        7020,  7080,  7140,  7200,  7260,  7320,  7380,  7440,  7500,
        7560,  7620,

In [169]:
player_time[player_time.match_id==23]

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
1051,23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1052,23,60,356,0,40,260,2,199,241,1,...,144,206,0,127,344,5,336,319,1,127
1053,23,120,456,0,65,360,2,395,464,4,...,552,306,0,158,444,5,428,556,4,662
1054,23,180,556,0,85,460,2,395,694,7,...,634,654,6,518,598,5,488,699,5,795
1055,23,240,699,1,435,695,4,603,998,8,...,634,1079,6,718,803,7,736,1199,5,1219
1056,23,300,958,1,663,1054,5,989,1374,14,...,845,1219,7,1064,1115,12,1076,1299,5,1270
1057,23,360,1095,2,1067,1320,9,1422,1640,18,...,1161,1485,11,1456,1368,16,1416,1399,5,1301
1058,23,420,1195,2,1200,1458,10,1628,1740,18,...,1846,2051,16,1961,1818,23,1950,1667,5,1579
1059,23,480,1607,3,1518,1682,13,1899,2028,18,...,2182,2446,21,2371,1918,23,2115,2101,6,1989
1060,23,540,2260,3,2060,2471,15,2605,2128,18,...,2369,3093,23,2714,2218,24,2280,2700,9,2394


In [162]:
tf

,match_id,start,end,last_death,deaths,radiant_gold,dire_gold,delta_gold,times,gold_t_0
0,0,220,252,237,3,632,790,-158,180,2
1,0,429,475,460,3,1929,345,1584,420,
2,0,900,936,921,3,3145,-506,3651,900,
3,0,1284,1328,1313,3,1872,-112,1984,1260,
4,0,1614,1666,1651,5,1768,2164,-396,1560,
...,...,...,...,...,...,...,...,...,...,...
539042,49999,1278,1323,1308,3,2142,783,1359,1260,
539043,49999,1394,1444,1429,3,1053,1680,-627,1380,
539044,49999,1920,1984,1969,4,942,3917,-2975,1920,
539045,49999,2304,2379,2364,7,-867,2936,-3803,2280,


In [135]:
tf.loc[0].i = player_time.loc[player_time['match_id'] == tf.loc[k].match_id, ('times','lh_t_1')].loc[player_time['times'] == tf.loc[k].times, 'lh_t_1'].iloc[0]

In [137]:
player_time.loc[player_time['match_id'] == tf.loc[k].match_id, ('times','lh_t_1')].loc[player_time['times'] == tf.loc[k].times, 'lh_t_1'].iloc[0]

9

In [148]:
tf.loc[0][i]

''

In [140]:
tf

,match_id,start,end,last_death,deaths,radiant_gold,dire_gold,delta_gold,times,gold_t_0
0,0,220,252,237,3,632,790,-158,180,
1,0,429,475,460,3,1929,345,1584,420,
2,0,900,936,921,3,3145,-506,3651,900,
3,0,1284,1328,1313,3,1872,-112,1984,1260,
4,0,1614,1666,1651,5,1768,2164,-396,1560,
...,...,...,...,...,...,...,...,...,...,...
539042,49999,1278,1323,1308,3,2142,783,1359,1260,
539043,49999,1394,1444,1429,3,1053,1680,-627,1380,
539044,49999,1920,1984,1969,4,942,3917,-2975,1920,
539045,49999,2304,2379,2364,7,-867,2936,-3803,2280,


In [79]:
event_times

0          220
1          429
2          900
3         1284
4         1614
          ... 
539042    1278
539043    1394
539044    1920
539045    2304
539046    2742
Name: start, Length: 539047, dtype: int64

In [128]:
player_time.loc[player_time['match_id'] == 0, ('times','lh_t_1')].loc[player_time['times'] == 120, 'lh_t_1'].iloc[0]

4

In [147]:
player_time.loc[idx]['lh_t_0']

0

In [115]:
idx = player_time.index[(player_time.match_id ==0) & (player_time.times == 60)].tolist()[0]

In [124]:
player_time

,match_id,times,gold_t_0,lh_t_0,xp_t_0,gold_t_1,lh_t_1,xp_t_1,gold_t_2,lh_t_2,...,xp_t_129,gold_t_130,lh_t_130,xp_t_130,gold_t_131,lh_t_131,xp_t_131,gold_t_132,lh_t_132,xp_t_132
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,60,409,0,63,142,1,186,168,0,...,62,345,6,351,100,0,77,613,1,125
2,0,120,546,0,283,622,4,645,330,0,...,543,684,12,805,200,0,210,815,5,323
3,0,180,683,1,314,927,9,1202,430,0,...,842,958,16,1135,300,0,210,1290,8,527
4,0,240,956,1,485,1264,11,1583,530,0,...,1048,1500,26,1842,400,0,210,1431,9,589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2209773,49999,2640,19098,215,20444,18261,129,27832,18988,89,...,24025,15992,91,16998,34374,296,32701,23114,290,22774
2209774,49999,2700,19460,227,20659,18478,134,28012,19108,90,...,24025,16092,91,16998,34474,296,32701,23214,290,22774
2209775,49999,2760,19948,237,20889,18889,148,28437,19208,90,...,25430,16518,91,17509,35434,305,32701,23602,290,24179
2209776,49999,2820,19997,237,20889,18938,148,28437,19257,90,...,28237,17246,92,18815,36272,306,32701,24181,294,24772


In [68]:
event_times

0          220
1          429
2          900
3         1284
4         1614
          ... 
539042    1278
539043    1394
539044    1920
539045    2304
539046    2742
Name: start, Length: 539047, dtype: int64

In [86]:
list(player_time.columns)

['match_id',
 'times',
 'gold_t_0',
 'lh_t_0',
 'xp_t_0',
 'gold_t_1',
 'lh_t_1',
 'xp_t_1',
 'gold_t_2',
 'lh_t_2',
 'xp_t_2',
 'gold_t_3',
 'lh_t_3',
 'xp_t_3',
 'gold_t_4',
 'lh_t_4',
 'xp_t_4',
 'gold_t_128',
 'lh_t_128',
 'xp_t_128',
 'gold_t_129',
 'lh_t_129',
 'xp_t_129',
 'gold_t_130',
 'lh_t_130',
 'xp_t_130',
 'gold_t_131',
 'lh_t_131',
 'xp_t_131',
 'gold_t_132',
 'lh_t_132',
 'xp_t_132']

In [43]:
for i in range(n_observations):
    # event_times for game i
    event_times = tf[tf.match_id == i].start
    
    # for each event_time, 
    for k in range(len(event_times)):
        for j in range(10):
            tf_player.at[10*k + j,'start'] = event_times.iloc[k]

In [38]:
event_times = tf[tf.match_id == 3].start

In [52]:
event_times = tf.start
len(event_times)

539047

In [53]:
len(tf_player)

5390470

In [45]:
x = tf[tf.match_id == 0]
y = x.start
y

0      220
1      429
2      900
3     1284
4     1614
5     1672
6     1734
7     1818
8     1863
9     2101
10    2248
11    2351
Name: start, dtype: int64

In [27]:
tf_player.head()

,match_id,player_slot,buybacks,damage,deaths,gold_delta,xp_end,xp_start,start
0,0,0,0,105,0,173,536,314,220
1,0,1,0,566,1,0,1583,1418,220
2,0,2,0,0,0,0,391,391,220
3,0,3,0,0,0,123,1775,1419,220
4,0,4,0,444,0,336,1267,983,220


In [29]:
tf_player.groupby(['match_id', 'start']).mean()

player_slot  buybacks  damage  deaths  gold_delta   xp_end  \
match_id start                                                               
0        220           66.0       0.0   222.8     0.3       142.2   1069.1   
         429           66.0       0.0   354.2     0.3       227.4   2204.6   
         900           66.0       0.0   345.9     0.3       263.9   5165.4   
         1284          66.0       0.0   907.3     0.3       176.0   7685.7   
         1614          66.0       0.0  1727.0     0.5       393.2  10563.4   
...                     ...       ...     ...     ...         ...      ...   
49999    1278          66.0       0.0   482.3     0.3       292.5   7959.3   
         1394          66.0       0.0   724.0     0.3       273.3   9016.0   
         1920          66.0       0.0   631.9     0.4       485.9  14807.9   
         2304          66.0       0.4  1878.4     0.7       206.9  20519.9   
         2742          66.0       0.0  1404.4     0.5       339.6  24696.4   

                xp_start  
match_id start            
0        220       885.2  
         429      1847.0  
         900      4681.2  
         1284     7284.3  
         1614     9542.4  
...                  ...  
49999    1278     7329.5  
         1394     8348.3  
         1920    13937.8  
         2304    18682.8  
         2742    23807.3  

[539047 rows x 7 columns]

In [30]:
tf_player.at[0,'start']=10

In [20]:
set_value?df.at[4, 'B']

Object `set_value` not found.
